In [1]:
from lib import *
import librosa.display
import matplotlib.pyplot as plt
import librosa
import os
from PIL import Image
import gradio as gr
from pydub import AudioSegment

In [2]:
classes = ["accapella", "accoustic", "electro"]

def vector_to_class_label(vector):
    max_index = np.argmax(vector)
    return classes[max_index]

def mp3_to_image(mp3_file, size):
    sacrifice_signal, sample_rate = librosa.load(mp3_file)
    librosa.display.waveshow(sacrifice_signal, sr=sample_rate)
    plt.savefig("out.png")


    image = Image.open("out.png").convert("L")

    image_resized = image.resize((size, size))

    img_array = np.array(image_resized)

    img_flattened = img_array.flatten()

    os.remove("out.png")

    return img_flattened

model = load_mlp_model(b"oxyd_alpha.model")

def audio_chunker(audio_file):
    size = 10
    audio = AudioSegment.from_file(audio_file)

    duration_ms = len(audio)

    chunk_duration_ms = 30 * 1000

    chunk = audio[:chunk_duration_ms]

    temp_file = "temp_chunk.mp3"
    chunk.export(temp_file, format="mp3")
    song_array = mp3_to_image(temp_file, size)


    prediction = predict_mlp_model(model, song_array, True, 3)
    prediction = vector_to_class_label(prediction)

    os.remove(temp_file)

    print(prediction)

    return str(prediction)


audio_input = gr.inputs.Audio(label="Sélectionnez un fichier audio", type="filepath")
output_prediction = gr.outputs.Textbox(label="Prédiction")

iface = gr.Interface(fn=audio_chunker, inputs=[audio_input], outputs=output_prediction)
iface.launch()

/tmp/ipykernel_3772806/2183540496.py:52: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  audio_input = gr.inputs.Audio(label="Sélectionnez un fichier audio", type="filepath")
/tmp/ipykernel_3772806/2183540496.py:52: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  audio_input = gr.inputs.Audio(label="Sélectionnez un fichier audio", type="filepath")
/tmp/ipykernel_3772806/2183540496.py:53: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  output_prediction = gr.outputs.Textbox(label="Prédiction")


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
